In [1]:
import torch
print(torch.cuda.is_available()) 
from Supporting_functions import *
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'        
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

True


# Predictions for a folder

In [2]:
# Define paths

modelName = 'Sinu'    #Nombre de la carpeta donde está el entrenamiento (estructura)
         #[deblur_net_98_10000, 97_5000, 96_5000, 94_5000, 93_5000, 90_10000, 89_5000, 86_10000, 85_10000, 84_10000]99_10000
iteration = ["98_5000"]        

def generar_model_path():
    for n in iteration:
        modelId = 'deblur_net_' + n + '.pkl'    #No hace falta tocar esto
        
        # Models
        #Directorio donde están las carpetas de entrenamiento
        srcpath = r'D:\Current_Segovia_Lab\Dilan_Martinez_temp\Selfnet\Regeneracion\Modelos/'+modelName+'/'    
        yield srcpath+'checkpoint/saved_models/'+modelId  

# Image to test
img_src_path = r'D:\Current_Segovia_Lab\Dilan_Martinez_temp\Selfnet\Regeneracion\Data\Nuevas deconvoluciones\self_Sinu'
scale=1

#output dir
outdir = rf'D:\Current_Segovia_Lab\Dilan_Martinez_temp\Selfnet\Regeneracion\Data\Nuevas deconvoluciones\Deconvoluciones\self_Sinu'

# CUDA device
device1 = torch.device('cuda:0')
device2 = torch.device('cuda:0')
batch_size = 16

# Image Normalization
min_v = 0
max_v = 65535
norm_percentiles = (30, 99.999)  #99.9995 For Nuclei, 99.999 For Membranes
thres_scale = 2 # threhold for simple mask for normalization

# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:


for model_path in generar_model_path():
    print(f"Using model: {model_path}")

    # Extraer solo el nombre del archivo sin la ruta
    filename = os.path.basename(model_path)  # 'deblur_net_98_10000.pkl'

    # Quitar la extensión .pkl
    filename_no_ext = filename[:-4]  # 'deblur_net_98_10000'

    # Encontrar la posición inicial y final del substring
    start_idx = filename_no_ext.rfind('_', 0, filename_no_ext.rfind('_')) + 1  # Índice del '98'
    end_idx = len(filename_no_ext)  # Final en '10000'

    # Extraer el substring
    n_m = filename_no_ext[start_idx:end_idx]  # '98_10000'
    
    # Prepare networks
    deblur_net_A = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
    deblur_net_B = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
    
    #deblur_net_A  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')
    #deblur_net_B  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')
    
    
     # Load Model
    deblur_net_A.load_state_dict(torch.load(model_path))
    deblur_net_B.load_state_dict(torch.load(model_path))
    
    
    # Get all tif images in the folder
    image_names = [f for f in os.listdir(img_src_path) if f.endswith('.tif')]
    
    # Calculate for each model
    for i, image_name in enumerate(image_names):
    
        start_time = time.time()  # Record the start time   
    
        # Open image
        img_path = os.path.join(img_src_path, image_name)
        img  =  65535 * tifffile.imread(img_path).astype(np.float32)
    
        # Normalize 
        img = image_normalization(img, norm_percentiles, thres_scale, min_v, max_v)
        img = img.astype(np.float32)
    
        # Predict image
        fusion_stack = upsample_block(img,scale,1,deblur_net_A,deblur_net_B,min_v,max_v, device1, device2,batch_size)
    
    
        # Image Normalization
        fusion_stack = image_normalization(fusion_stack, norm_percentiles, thres_scale, min_v, max_v)
    
        # Save image
        img_out = os.path.join(outdir, f'{n_m}_{image_name}')
        tifffile.imwrite(img_out,fusion_stack)
    
        elapsed_time = time.time() - start_time
        print(f"Elapsed Time: {elapsed_time:.4f} seconds, image {image_name}")   
    
    del deblur_net_A
    del deblur_net_B


Using model: D:\Current_Segovia_Lab\Dilan_Martinez_temp\Selfnet\Regeneracion\Modelos/Sinu/checkpoint/saved_models/deblur_net_98_5000.pkl
initialize network with normal
initialize network with normal


C:\Users\WORKSTATION-PC\AppData\Local\Temp\ipykernel_28696\4202193926.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  deblur_net_A.load_state_dict(torch.load(model_path

mode_result.mode :  131070.0
     -threshold_value: 262140.0
Intensity Norm  from (0 , 4189390336) to  (0, 65535)  



Predicting : 100%|██████████████████████████████████████████████████████████████| 101/101 [01:03<00:00,  1.59 planes/s]


mode_result.mode :  25
     -threshold_value: 50
Intensity Norm  from (0 , 37037) to  (0, 65534)  

Elapsed Time: 295.0775 seconds, image C2-20220418_3_02_corrected.tif
mode_result.mode :  196605.0
     -threshold_value: 393210.0
Intensity Norm  from (0 , 4207674624) to  (0, 65535)  



Predicting : 100%|██████████████████████████████████████████████████████████████| 101/101 [00:56<00:00,  1.78 planes/s]


mode_result.mode :  30
     -threshold_value: 60
Intensity Norm  from (0 , 36473) to  (0, 65535)  

Elapsed Time: 263.5186 seconds, image C2-20220418_3_04_corrected.tif
mode_result.mode :  131070.0
     -threshold_value: 262140.0
Intensity Norm  from (0 , 4228383744) to  (0, 65535)  



Predicting :  90%|████████████████████████████████████████████████████████      | 47/52 [1:05:07<06:55, 83.12s/ planes]

In [ ]:
print(img_out)

In [ ]:
#tifffile.imwrite(img_out,fusion_stack)